# Azure ML - Jupyter Notebooks

## Loguearse a Azure ML

Como las acciones que vamos a hacer por CLI o a través del SDK de Python necesitan una autentificación, primero vamos a loguearnos en Azure ML

### Login en Azure ML con el CLI de Azure ML

Para logearnos en Azure hacemos

In [ ]:
!az login

Se nos abrirá el navegador para logearnos

### Crear un cliente de Azure ML con el SDK de Python

Primero creamos dos variables con la ID de la suscripción y el grupo de recursos, como estos son datos personales, no los voy a poner aquí. Lo que voy a hacer es incluirlos en un archivo `.env` que no voy a subir a GitHub

```bash
AZURE_SUSCRIPION_ID="xxxxx-xxxx-xxxx-xxxx-xxxxx"
AZURE_ML_RESOURCE_GRPU_ID="xxxxx-xxxx-xxxx-xxxx-xxxxx"
```

Ahora para leerlos primero necesitasos tener instalado `dotenv` que lo hacemos mediante `pip install python-dotenv`

In [1]:
import os
import dotenv

dotenv.load_dotenv()

AZURE_SUSCRIPION_ID = os.getenv("AZURE_SUSCRIPION_ID")
AZURE_ML_RESOURCE_GRPU_ID = os.getenv("AZURE_ML_RESOURCE_GRPU_ID")


Ahora que tenemos estas variables creamos un cliente

In [2]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

workspace_name = "azure-ml-workspace-Python-SDK"

ml_client = MLClient(DefaultAzureCredential(), AZURE_SUSCRIPION_ID, AZURE_ML_RESOURCE_GRPU_ID, workspace_name)

## Scripts

Una vez que hemos validado el código de entrenamiento en un Jupyter Notebook podemos pasarlo a un script para ejecutarlo y poder hacer varios experimentos cambiando hiperparámetros

### Ejecutar un script desde la interfaz gráfica

#### Subir los datos

Nos vamos a `Notebooks` en la zona de las carpetas hay un botón con el símbolo `+`, si le damos podemos subir archivos. Vamos a subir la carpeta `en` que descargamos de HuggingFace, marcamos la casilla que dice si creemos en los autores y le damos a `Upload`. Ya tenemos los datos subidos

#### Crear el script

Volvemos a dar al botón con el símbolo `+`, le damos a `Create new file`, en `File type` seleccionamos `Python (*.py)` y le ponemos el nombre `text-classification.py`. Ahora copiamos el siguiente código

```python
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import argparse

tokenizer = None

def load_datasets(folder_path, train_file, validation_file, test_file):
    dataset_train = load_dataset("json", data_files=f"{folder_path}/{train_file}")
    dataset_validation = load_dataset("json", data_files=f"{folder_path}/{validation_file}")
    dataset_test = load_dataset("json", data_files=f"{folder_path}/{test_file}")
    return dataset_train, dataset_validation, dataset_test

def get_dataset_num_classes(dataset, label):
    return len(dataset[label].unique('label'))

def get_tokenizer(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token
    return tokenizer

def tokenize_function(examples, max_length=768):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=max_length, return_tensors="pt")

def get_model(model_name, num_classes):
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_classes)
    model.config.pad_token_id = model.config.eos_token_id
    return model

class ReviewsDataset(Dataset):
    def __init__(self, huggingface_dataset):
        self.dataset = huggingface_dataset

    def __getitem__(self, idx):
        label = self.dataset[idx]['label']
        input_ids = torch.tensor(self.dataset[idx]['input_ids'])
        attention_mask = torch.tensor(self.dataset[idx]['attention_mask'])
        return input_ids, attention_mask, label

    def __len__(self):
        return len(self.dataset)

def predicted_labels(logits):
    percent = torch.softmax(logits, dim=1)
    predictions = torch.argmax(percent, dim=1)
    return predictions

def compute_accuracy(logits, labels):
    predictions = predicted_labels(logits)
    correct = (predictions == labels).float()
    return correct.mean()

def main(checkpoints, percentage_train, percentage_val_test, BS, LR, EPOCHs):
    global tokenizer

    dataset_train, dataset_validation, dataset_test = load_datasets("en", "train.jsonl", "en_validation.jsonl", "en_test.jsonl")
    num_classes = get_dataset_num_classes(dataset_train, "train")

    tokenizer = get_tokenizer(checkpoints)

    dataset_train = dataset_train.map(tokenize_function, batched=True, remove_columns=['id', 'label_text'])
    dataset_validation = dataset_validation.map(tokenize_function, batched=True, remove_columns=['id', 'label_text'])
    dataset_test = dataset_test.map(tokenize_function, batched=True, remove_columns=['id', 'label_text'])

    subset_train = dataset_train['train'].select(range(int(len(dataset_train['train']) * percentage_train)))
    subset_validation = dataset_validation['train'].select(range(int(len(dataset_validation['train']) * percentage_val_test)))
    subset_test = dataset_test['train'].select(range(int(len(dataset_test['train']) * percentage_val_test)))

    model = get_model(checkpoints, num_classes)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.half().to(device)
    print(f"device: {device}")

    train_dataset = ReviewsDataset(subset_train)
    validatation_dataset = ReviewsDataset(subset_validation)
    test_dataset = ReviewsDataset(subset_test)

    train_loader = DataLoader(train_dataset, batch_size=BS, shuffle=True)
    validation_loader = DataLoader(validatation_dataset, batch_size=BS)
    test_loader = DataLoader(test_dataset, batch_size=BS)

    optimizer = AdamW(model.parameters(), lr=LR)

    accuracy = 0
    for epoch in range(EPOCHs):
        model.train()
        train_loss = 0
        progresbar = tqdm(train_loader, total=len(train_loader), desc=f'Epoch {epoch + 1}')
        for input_ids, at_mask, labels in progresbar:
            input_ids = input_ids.to(device)
            at_mask = at_mask.to(device)
            label = labels.to(device)
            output = model(input_ids=input_ids, attention_mask=at_mask, labels=label)
            loss = output['loss']
            train_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            progresbar.set_postfix({'train_loss': loss.item()})
        train_loss /= len(train_loader)
        progresbar.set_postfix({'train_loss': train_loss})
        model.eval()
        valid_loss = 0
        progresbar = tqdm(validation_loader, total=len(validation_loader), desc=f'Epoch {epoch + 1}')
        for input_ids, at_mask, labels in progresbar:
            input_ids = input_ids.to(device)
            at_mask = at_mask.to(device)
            labels = labels.to(device)
            output = model(input_ids=input_ids, attention_mask=at_mask, labels=labels)
            loss = output['loss']
            valid_loss += loss.item()
            step_accuracy = compute_accuracy(output['logits'], labels)
            accuracy += step_accuracy
            progresbar.set_postfix({'valid_loss': loss.item(), 'accuracy': step_accuracy.item()})
        valid_loss /= len(validation_loader)
        accuracy /= len(validation_loader)
        progresbar.set_postfix({'valid_loss': valid_loss, 'accuracy': accuracy})
    print(f'Accuracy: {accuracy}')

    # Save the model
    model.save_pretrained("model")
    tokenizer.save_pretrained("model")

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--checkpoints", type=str, default="openai-community/gpt2")
    parser.add_argument("--percentage_train", type=float, default=0.0001)
    parser.add_argument("--percentage_val_test", type=float, default=0.001)
    parser.add_argument("--batch_size", type=int, default=2)
    parser.add_argument("--learning_rate", type=float, default=2e-5)
    parser.add_argument("--epochs", type=int, default=1)
    args = parser.parse_args()

    checkpoints = args.checkpoints
    percentage_train = args.percentage_train
    percentage_val_test = args.percentage_val_test
    batch_size = args.batch_size
    learning_rate = args.learning_rate
    epochs = args.epochs

    main(checkpoints, percentage_train, percentage_val_test, batch_size, learning_rate, epochs)
```

#### Ejecutar el script

Volvemos a dar al botón con el símbolo `+`, le damos a `Create new file`, en `File type` seleccionamos `Notebook (*.ipynb)` y le ponemos el nombre `run_text-clasification.ipynb`. Ahora copiamos las siguientes celdas de código

```python
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

workspace_name = "azure-ml-workspace-Python-SDK"

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except:
    credential = InteractiveBrowserCredential()
```

```python
ml_client = MLClient.from_config(credential=credential)
```

```python
from azure.ai.ml import command

execute_command = "python text-clasification.py --learning_rate 2e-5"
environment = "cuda_11_8_0_cudnn8_devel_ubuntu22_04_transformers_GUI@latest"
compute_instance = "compute-instance-GUI"
display_name = "text-clasificator"
experiment_name = "train-classification-model"

# configure job
job = command(
    code="./",
    command=execute_command,
    environment=environment,
    compute=compute_instance,
    display_name=display_name,
    experiment_name=experiment_name
)
```

```python
returned_job = ml_client.create_or_update(job)
```

Mira que en el environment hemos puesto `@latest` para que use la última versión del entorno

Otra cosa importate es que los hiperparámetros como el batch size, el learning rate, etc. se pasan al script como argumentos, por lo que puedes hacer muchos experimentos modificando estos valores en los argumentos del script

Seleccionamos la compute instance `compute-instance-GUI` y ejecutamos todas las celdas, si todo ha ido bien, se habrá creado un nuevo experimento en Azure ML

#### Visualizar el entrenamiento

Si nos vamos a la sección `Jobs` en la parte izquierda de la pantalla, veremos el experimento que hemos creado, si le damos veremos la salida del script. Al igual que con los notebooks no he conseguido poder ejecutar el script en una GPU, por eso en el código se crea un subconjunto del dataset tan pequeño, por eso el batch size es tan pequeño y el número de épocas es solo una. Por lo que tardará bastante en terminar

### Ejecutar un script con el CLI de Azure ML

No he encotrado la manera de ejecutar el script mediante la CLI de Azure ML

### Ejecutar un script con el SDK de Python

Para ejecutar un script primero tenemos que subir los datos y el script igual que lo hemos hecho en la interfaz gráfica. No he encotrado la manera de subirlos mediante el SDK de Python. Por lo que nos aseguramos de estar en el workspace en el que hemos hecho todo con el SDK de Python y subimos los datos y el script igual que lo hemos hecho antes

Ahora para ejecutar ek script ejecutamos ek mismo código que ejecutamos en el notebook mediante la interfaz gráfica

In [3]:
from azure.ai.ml import command

execute_command = "python text-clasification.py --learning_rate 2e-5"
environment = "cuda_11_8_0_cudnn8_devel_ubuntu22_04_transformers_python_@latest"
compute_instance = "compute-instance-Python"
display_name = "text-clasificator"
experiment_name = "train-classification-model"

# configure job
job = command(
    code="./",
    command=execute_command,
    environment=environment,
    compute=compute_instance,
    display_name=display_name,
    experiment_name=experiment_name
)

In [5]:
returned_job = ml_client.create_or_update(job)